# Data preprocessing

fix bug with paths in test.yaml

In [ ]:
!cp data/test.yaml test.yaml
!sed -i 's/\/net\/pal-soc1.us.bosch.com\/ifs\/data\/Shared_Exports\/deep_learning_data\/traffic_lights\/university_run1/.\/data\/rgb\/test/g' test.yaml

create tfrecords to speedup training

In [ ]:
! python3 bstld/tf_object_detection/to_tfrecords.py \
   --dataset_folder data \
   --train_yaml data/train.yaml \
   --test_yaml test.yaml \
   --additional_yaml data/additional_train.yaml

# Train a model

In [ ]:
! python3 /root/models/research/object_detection/legacy/train.py \
  -train_dir=trained -pipeline_config_path=ssd_mobilenet_v1.config

# Evaluate a model

In [ ]:
! python3 bstld/tf_object_detection/model_freezer.py \
  --train_dir trained

apply model

In [ ]:
#! python3 bstld/tf_object_detection/inference.py \
#  --frozen_graph trained/model.ckpt-128199_proto/frozen_inference_graph.pb \
#  --data_input test.yaml \
#  --out_folder /tmp/4

from bstld.tf_object_detection.inference import label_file_inference

graph_path = "trained/model.ckpt-128199_proto/frozen_inference_graph.pb"
data_file = "test.yaml"
threshold = .05
results = label_file_inference(graph_path,
                               data_file,
                               json_out_path='',
                               show=0,
                               out_folder=None,
                               threshold=threshold)


compute metrics

In [ ]:
from bstld.evaluation.evaluate import evaluate

with open('test.yaml', 'r') as f:
    gt = yaml.load(f)
    
res = evaluate(results, gt, "res.json")